# Les templates d'email

Les emails transactionnels sont envoyés en utilisant le système de template Brevo.

Vous trouvez ici l'objectif de chaque email, ainsi que les paramètres passés en exécutant les briques de code.

In [ ]:
# init Django and get site params

from django.conf import settings
from django.db.models import Model

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["DJANGO_SETTINGS_MODULE"] = "recoco.settings.development"

import django
django.setup()

from recoco.apps.home.models import SiteConfiguration
site_config = SiteConfiguration.objects.first()
site = site_config.site
settings.SITE_ID = site.pk

from recoco.apps.communication.api import get_site_params

print("Paramètres lié au site")
print(get_site_params())

## "Projet bien reçu" et "Projet accepté par l'équipe de modération"
`TPL_PROJECT_RECEIVED` et `TPL_PROJECT_ACCEPTED`

In [ ]:
import pprint

from recoco.apps.communication import digests
from recoco.apps.projects.models import Project

project = Project.objects.first()
params = {
    "project": digests.make_project_digest(project, project.owner),
}
pprint.pprint(params, compact=True)


## Le template "Résumé quotidien général de notifications"
`TPL_DIGEST_FOR_NON_SWITCHTENDER`

In [ ]:
from django.conf import settings
from django.contrib.auth import models as auth_models
from django.db import transaction
from model_bakery import baker
from notifications.signals import notify
from recoco.apps.communication.digests import send_digest_for_non_switchtender_by_user
from recoco.apps.tasks import models as task_models
from recoco import verbs

assert settings.DEBUG

try:
    with transaction.atomic():
        task = baker.make(task_models.Task, project=project, public=True)

        notify.send(
                sender=project.owner,
                recipient=project.owner,
                verb=verbs.Recommendation.DONE,
                action_object=task,
                target=project,
                public=True,
        )

        send_digest_for_non_switchtender_by_user(project.owner)

        raise Exception("no db change")
except:
    pass

## Le template "Résumé quotidien des conseillers"
`TPL_DIGEST_FOR_SWITCHTENDER`

In [ ]:
from django.conf import settings
from recoco.apps.communication.digests import send_digest_for_switchtender_by_user

assert settings.DEBUG

try:
    with transaction.atomic():
        task = baker.make(task_models.Task, project=project, public=True)

        notify.send(
                sender=project.owner,
                recipient=project.owner,
                verb=verbs.Recommendation.DONE,
                action_object=task,
                target=project,
                public=True,
        )

        send_digest_for_switchtender_by_user(project.owner)

        raise Exception("no db change")
except:
    pass

## Le template "Résumé des nouvelles recommandations"
`TPL_NEW_RECOMMENDATIONS_DIGEST`

In [ ]:
from django.contrib.contenttypes.models import ContentType
from recoco.apps.communication.digests import make_digest_of_project_recommendations_from_notifications 


try:
    with transaction.atomic():
        task = baker.make(task_models.Task, project=project, public=True)
        notify.send(
                sender=project.owner,
                recipient=project.owner,
                verb=verbs.Recommendation.DONE,
                action_object=task,
                target=project,
                public=True,
        )

        project_ct = ContentType.objects.get_for_model(Project)
        project_notifications = project.owner.notifications.filter(
            target_content_type=project_ct.pk,
            target_object_id=project.pk,
        )

        params = make_digest_of_project_recommendations_from_notifications(project, project_notifications, project.owner)
        pprint.pprint(params)

        raise Exception("no db change")
except:
    pass

## Le template "Alerte conseillers d'un nouveau projet sur le territoire"
`TPL_NEW_SITE_FOR_SWITCHTENDER`

In [ ]:
from recoco.apps.communication.digests import send_digests_for_new_sites_by_user

try:
    with transaction.atomic():
        notify.send(
                sender=project.owner,
                recipient=project.owner,
                verb=verbs.Project.AVAILABLE,
                action_object=project,
                target=project,
                public=True,
        )

        send_digests_for_new_sites_by_user(project.owner)

        raise Exception("no db change")
except:
    pass

## Le template "Invitation à rejoindre un projet"
`TPL_SHARING_INVITATION`

In [ ]:
from django.conf import settings
from model_bakery import baker
from recoco.apps.invites import models as invites_models
from recoco.apps.invites.api import invite_send

assert settings.DEBUG

invite = baker.prepare(invites_models.Invite, site=site, project=project, inviter=project.owner)
invite_send(invite)

## Le template "Rappel des nouvelles recommandations (mail projet B)" et Le template "Où en êtes-vous ? (mail projet C)"
`TPL_PROJECT_REMINDERS_NEW_RECO_DIGEST` et `TPL_PROJECT_REMINDERS_WHATS_UP_DIGEST`

In [ ]:
from recoco.apps.communication.digests import make_digest_of_project_recommendations
from recoco.apps.tasks.models import Task

task = baker.prepare(Task, site=site, project=project, created_by=project.owner)
tasks = [task]

params = make_digest_of_project_recommendations(project, tasks, project.owner)
pprint.pprint(params)